# ENGR 240: Parameters in Roots Problems - Michaelis-Menten Kinetics

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%203/Worksheet%203-2_template.ipynb)

## Introduction

In many engineering applications, we encounter nonlinear equations that need to be solved for different parameter values. This worksheet focuses on how to handle changing parameters when using root-finding methods and explores the concept of "residuals" in numerical approximations.

### Learning Objectives
- Implement parameter-passing techniques in root-finding methods
- Compare Newton-Raphson method and `scipy.root_scalar` for parametric roots problems
- Understand and calculate residuals in numerical approximations
- Visualize substrate concentration profiles using the Michaelis-Menten model
- Gain programming practice with functions, loops, and arrays

### Mathematical Background

The **Michaelis-Menten model** describes the kinetics of enzyme-mediated reactions. It is represented by the following equation:

$$S = S_0 - v_m t + k_s \ln\left(\frac{S_0}{S}\right)$$

where:
- $S$ = substrate concentration (mol/L)
- $S_0$ = initial substrate concentration (mol/L) at t = 0
- $v_m$ = maximum uptake rate (mol/L/d)
- $k_s$ = half saturation constant (mol/L)
- $t$ = time (days)

For this worksheet, we need to solve for $S$ given specific values of $S_0$, $v_m$, $k_s$, and $t$. Since $S$ appears on both sides of the equation, we can't solve for it directly. Instead, we need to rearrange the equation to find the roots:

$$f(S) = S - S_0 + v_m t - k_s \ln\left(\frac{S_0}{S}\right) = 0$$

## Setup and Imports

First, let's import the necessary libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
import time  # for timing comparison

## Newton-Raphson Method with Parameters

The Newton-Raphson method is an iterative approach to finding roots. We've previously studied the basic implementation. The method uses the formula:

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$

When working with parameters, we need to modify our implementation to accept additional parameter values.

### Implementation with Parameter Passing

Let's define our function and its derivative for the Michaelis-Menten model:

In [ ]:
def mm_function(S, S0, vm, ks, t):
    """
    Michaelis-Menten function whose root we seek.

    Parameters:
    -----------
    S : float
        Substrate concentration (mol/L)
    S0 : float
        Initial substrate concentration (mol/L)
    vm : float
        Maximum uptake rate (mol/L/d)
    ks : float
        Half saturation constant (mol/L)
    t : float
        Time (days)

    Returns:
    --------
    float : Function value at S
    """
    return S - S0 + vm * t - ks * np.log(S0 / S)

### Task 1: Derive and Implement the Derivative Function

Derive the derivative of the Michaelis-Menten function with respect to S:

$$\frac{d}{dS}\left[S - S_0 + v_m t - k_s \ln\left(\frac{S_0}{S}\right)\right]$$

Implement your derived formula in the `mm_derivative` function below. Test your implementation by running the Newton-Raphson method.

In [ ]:
# NOTE: You will need to derive and implement the derivative function.
# Remove the next line and add your implementation for mm_derivative

"""
STUDENT ACTIVITY:
1. Derive the derivative of mm_function with respect to S
2. Implement the derivative function below
"""

def mm_derivative(S, S0, vm, ks, t):
    """
    Derivative of the Michaelis-Menten function with respect to S.

    Parameters:
    -----------
    (Same as mm_function)

    Returns:
    --------
    float : Derivative value at S
    """
    # Replace this with your implementation
    pass

Now let's implement the Newton-Raphson method with parameter passing:

In [ ]:
def newton_raphson(func, dfunc, x0, tol=1e-8, max_iter=100, **params):
    """
    Newton-Raphson method to find a root of a function.

    Parameters:
    -----------
    func : function
        Function to find root of
    dfunc : function
        Derivative of the function
    x0 : float
        Initial guess
    tol : float
        Error tolerance
    max_iter : int
        Maximum number of iterations
    **params : dict
        Additional parameters to pass to the function

    Returns:
    --------
    tuple : (root, iterations, error, residual)
    """
    x = x0
    error = float('inf')
    iter_count = 0

    while error > tol and iter_count < max_iter:
        f_x = func(x, **params)
        df_x = dfunc(x, **params)

        if df_x == 0:
            raise ValueError("Derivative is zero. Cannot continue.")

        x_new = x - f_x / df_x

        # Calculate relative error
        error = abs((x_new - x) / x_new) if x_new != 0 else abs(x_new - x)

        x = x_new
        iter_count += 1

    # Calculate residual - the function value at our solution
    residual = abs(func(x, **params))

    return x, iter_count, error, residual

## Using scipy.root_scalar with Parameters

SciPy's `root_scalar` function is a powerful tool for finding roots of nonlinear equations. The recommended Brentq method uses an optimized combination of bisection (for reliability) and inverse quadratic interpolation (for speed). This makes it robust for a wide range of problems.

### How brentq Works

The `fsolve` function is designed to solve systems of nonlinear equations. Its algorithm works as follows:

1. It approximates the Jacobian matrix (which contains derivatives) using finite differences
2. Uses this approximation to determine a search direction
3. Takes steps in that direction, adapting the step size as needed
4. Continues iterating until convergence or reaching the maximum iterations

### Basic Syntax and Parameters

The basic syntax of `fsolve` is:

```python
result = optimize.fsolve(func, x0, args=(), xtol=1e-08, maxfev=0, ...)
```

Where:
- `func` is the function to find the roots of
- `x0` is the initial guess
- `args` is a tuple of arguments that will be passed to the function (this is for passing parameters to evaluate your function, not algorithm parameters)
- `xtol` is the relative error tolerance (an algorithm parameter)
- `maxfev` is the maximum number of function evaluations (an algorithm parameter)

#### Algorithm Parameters vs. Function Parameters

It's important to distinguish between two types of parameters:

1. **Algorithm parameters** - These control how `fsolve` operates, such as:
   ```python
   # Example of adjusting algorithm parameters
   result = optimize.fsolve(func, x0, xtol=1e-10, maxfev=1000, full_output=True)
   ```
   
2. **Function parameters** - These are passed to your function to evaluate it (what this worksheet focuses on):
   ```python
   # Example of passing parameters to your function
   def my_func(x, a, b, c):
       return x**2 + a*x + b - c
       
   # These parameters (5, 3, 10) are passed to my_func as a, b, c
   result = optimize.fsolve(my_func, x0=1.0, args=(5, 3, 10))
   ```

#### When to Use args vs. Alternative Parameter Passing Approaches

You might wonder: If `args` can pass parameters to our function, why explore more complex approaches?

The `args` parameter in `fsolve` provides a simple way to pass parameters:

```python
# Using args for parameter passing
result = optimize.fsolve(my_func, x0=1.0, args=(5, 3, 10))
```

However, there are several reasons to consider alternative approaches:

1. **Keyword vs. positional arguments**: The `args` approach only passes positional arguments. Our MM function uses keyword arguments (`S0=`, `vm=`, etc.) which are more readable.

2. **Parameter order dependency**: With `args`, you must maintain the exact parameter order as defined in the function signature.

3. **Changing parameters in loops**: When solving for many different parameter combinations (like our time loop), rebuilding the `args` tuple each time can be cumbersome.

4. **Readability**: Lambda functions and wrappers can make the code's intent clearer, especially when only some parameters change while others remain fixed.

For this worksheet, we'll explore alternative parameter passing strategies that provide more flexibility and maintainability.

Let's see how to implement the first approach:

In [ ]:
def solve_with_fsolve(t, S0, vm, ks, x0):
    """
    Solve the Michaelis-Menten equation using scipy.fsolve.

    Parameters:
    -----------
    t : float
        Time (days)
    S0, vm, ks : float
        Model parameters
    x0 : float
        Initial guess

    Returns:
    --------
    tuple : (root, residual)
    """
    # Lambda function with parameters
    f = lambda S: mm_function(S, S0=S0, vm=vm, ks=ks, t=t)

    # Solve using fsolve
    root = optimize.fsolve(f, x0)[0]

    # Calculate residual
    residual = abs(f(root))

    return root, residual

## Problem Setup

Let's set up our problem with the following parameters:
- Initial substrate concentration (S0) = 10 mol/L
- Maximum uptake rate (vm) = 0.5 mol/L/d
- Half saturation constant (ks) = 2 mol/L
- Time range from 0 to 50 days in steps of 5 days
- Use a stopping tolerance of 1e-8 for both methods

In [ ]:
# Set parameters
S0 = 10.0  # mol/L
vm = 0.5   # mol/L/d
ks = 2.0   # mol/L
t_values = np.arange(0, 51, 5)  # days

# Initial guess for S
initial_guess = 0.0001

# Define parameter dictionaries for both methods
newton_params = {'S0': S0, 'vm': vm, 'ks': ks}
fsolve_bracket = [0.0001, 10.01]  # Initial bracket for fsolve

## Calculating S vs t and Residuals

Now, let's calculate S vs t and the associated residuals for both methods.

### Task 2: Implement Parameter Looping

For this task, you need to:

1. Create arrays to store the results
2. Loop through each time value
3. Call both root-finding methods with the appropriate parameters
4. Calculate and store the residuals
5. Compare the computation times

Start by setting up empty arrays to store your results and then implement a loop to call both root-finding methods for each time value. Your implementation should follow this general structure:

```python
# Arrays to store results
# Your code here

# Time execution
# Your code here

# Loop through each time value
# Your code here

# Print computation times
# Your code here
```

In [ ]:
# Your implementation here

## Visualizing the Results

### Task 3: Create Visualizations

Create plots to visualize your results. You should:

1. Plot S vs t for both methods
2. Plot the residuals vs t for both methods

Your implementation should produce informative plots with appropriate labels, titles, and legends.

In [ ]:
# Your visualization code here

## Discussion Questions

After completing the implementation and visualization tasks, consider the following questions:

1. How do the two methods compare in terms of:
   - Accuracy (residuals)
   - Computational efficiency (execution time)
   - Ease of implementation

2. How would you modify the code to handle multiple parameter variations (not just time) efficiently?

## Conclusion

In this worksheet, we've explored how to handle parameters in root-finding problems using the Newton-Raphson method and `scipy.fsolve`. We've also examined the concept of residuals in numerical approximations and compared the efficiency and accuracy of different approaches.

Key takeaways:
1. Parameter passing in numerical methods can be handled through function arguments or keyword arguments
2. Lambda functions provide a clean way to parameterize functions for `scipy.fsolve`
3. Residuals help us understand the accuracy of our numerical solutions
4. For complex problems, specialized libraries like SciPy offer robust and efficient solutions

